In [3]:
import itertools

from trec_car.read_data import iter_annotations

from tools.dataloaders import WikipediaCBOR, PageFormat

import pandas as pd

"""
from pyspark.sql.dataframe import DataFrame
from pyspark.sql.column import Column
from pyspark.sql.types import StructType, ArrayType, StringType, DataType

from pyspark.sql import SparkSession, SQLContext


spark = SparkSession.builder.appName("colla-tokenizer").getOrCreate()
sc = spark.sparkContext
sqlContext = SQLContext(sc)
"""

'\nfrom pyspark.sql.dataframe import DataFrame\nfrom pyspark.sql.column import Column\nfrom pyspark.sql.types import StructType, ArrayType, StringType, DataType\n\nfrom pyspark.sql import SparkSession, SQLContext\n\n\nspark = SparkSession.builder.appName("colla-tokenizer").getOrCreate()\nsc = spark.sparkContext\nsqlContext = SQLContext(sc)\n'

In [4]:
wikipedia_cbor = WikipediaCBOR("wikipedia/car-wiki2020-01-01/enwiki2020.cbor",
                                "wikipedia/car-wiki2020-01-01/partitions",
                                # repreprocess=True, page_lim=1000
                                )

Loaded from cache


In [5]:

# a = wikipedia_cbor[0]
k = 5
with open(wikipedia_cbor.cbor_path, "rb") as cbor_file:
    topk = itertools.islice(iter_annotations(cbor_file), k)
    topk = map(wikipedia_cbor.preprocess_page, enumerate(topk))

    df = pd.DataFrame(topk)
    #df = sqlContext.createDataFrame(topk)
    # page = next(iter_annotations(cbor_file))
    # dataloader_page = wikipedia_cbor.preprocess_page(page)
    # spacy_collect_mentions(dataloader_page)


In [6]:
df.head()

,id,title,text,link_mentions
0,0,Anarchism,Anarchism is an anti-authoritarian political a...,"[(3818014, 16, 33), (4210689, 35, 43), (540829..."
1,1,Albedo,"Albedo () (, meaning 'whiteness') is the measu...","[(3849449, 56, 73), (6970944, 78, 92), (612884..."
2,2,Autism,Autism is a developmental disorder characteriz...,"[(5748136, 12, 33), (215297, 313, 321), (57595..."
3,3,A,A or a is the first letter and the first vowel...,"[(4796107, 20, 25), (4772344, 41, 52), (435768..."
4,4,Achilles,"In Greek mythology, Achilles ( ) or Achilleus ...","[(4207422, 3, 17), (5079607, 69, 78), (6013052..."


In [7]:
import spacy

nlp = spacy.load("xx_ent_wiki_sm")

In [32]:
from typing import List, Tuple
#from pyspark.sql.functions import pandas_udf
from spacy.attrs import LOWER, ENT_TYPE
import numpy as np
import pandas as pd

from trie_search import RecordTrieSearch
from heapq import merge

def autolink(page_id: int, title: str, text: str, links: List[Tuple[int, int, int]]):
    title_toks = nlp(title)
    toks = nlp(text)
    # ents = toks.ents

    link_idx = 0

    exact_mentions = {}
    mention_builder = []

    link = None

    
    
    # TODO: deal with ambiguities...
    exact_mentions[" ".join(str(tok) for tok in title_toks)] = page_id

    for tok in toks:
        # the new token ends a mention: register the mention
        if link_idx < len(links):
            link = links[link_idx]

        if link is not None and tok.idx > link[2] and len(mention_builder) > 0:
            exact_mentions[" ".join(mention_builder)] = link[0]
            mention_builder = []

        while link_idx < len(links) - 1 and tok.idx > link[2]:
            link_idx += 1
            link = links[link_idx]
        
        if link_idx >= len(links):
            link = None
        
        elif tok.idx >= link[1]:
            mention_builder.append(str(tok))
    
    # NOTE: it does use spaces as separators therefore any kind of punctuation ruins the trie's efficacy.
    # Would be better to run this on pretokenized text, but links would have to be recomputed.
    trie = RecordTrieSearch("<Q", map(lambda x: (x[0], (x[1],)), exact_mentions.items()))

    # print(trie.items())
    patterns = sorted(trie.search_longest_patterns(text), key=lambda x: x[1]) # sort by apparison

    link = None
    link_idx = 0
    new_links = []

    for title, new_link_id, idx in patterns:
        new_link = (new_link_id, idx, idx + len(title) - 1)

        if link_idx < len(links):
            link = links[link_idx]
        
        while link_idx < len(links) - 1 and idx > link[2]:
            link_idx += 1
            link = links[link_idx]

        if link_idx >= len(links):
            link = None
        
        if link is None or idx < link[1]:
            new_links.append(new_link)
    
    # return exact_mentions

    return list(merge(links, new_links, key=lambda x: x[1]))
    

In [33]:
autolink(0, "Anarchy", *list(df.iloc[0])[2:])

In [10]:
from trie_search import RecordTrieSearch

patterns = [("Albert Einstein", (1,)), ("Einstein", (1,)), ("Albert Hitchcock", (2,))]
t = RecordTrieSearch("<Q", patterns)

In [11]:
t.get("Albert Einstein")

[(1,)]